# Areas of Interest Data Analysis

In [1]:
# lib for dataframes
import pandas as pd

from scipy import stats

from chord import Chord

Config

In [2]:
config_datasheet = r'./results/data_of_all_removed.xlsx'

Import Dataframe

In [3]:
df_array = []
df = pd.read_excel(config_datasheet)

Prepare Image Paths

In [4]:
algo_name_array = df["Algorithm"].unique()
aoi_cat_path_array = []

for algo_name in algo_name_array:
    aoi = 'data/aoi_categorized/AOI_' + algo_name + '.xlsx'
    aoi_cat_path_array.append(aoi)
    df_array.append(df.loc[df["Algorithm"]==algo_name])

Load AOI to dataframe

In [5]:
aoi_df_array = []
for path in aoi_cat_path_array:
    raw = pd.read_excel(path)
    tmp_df = pd.DataFrame(raw)
    aoi_df_array.append(tmp_df)

Cast Data to Valid format

In [6]:
#cast data to valid format
def to_valid(string_points: str):
    string_points = string_points.strip()
    coordinate_str_array = string_points.split(" ")
    coordinates = []
    for coordinate_str in coordinate_str_array:
        coordinate = coordinate_str.split("-")
        coordinate = (int(coordinate[1]))
        coordinates.append(coordinate) 
    return coordinates

In [7]:
aoi_names = ['none', 'main', 'definition', 'pre calculation', 'if condition', 'if step', 'iteration condition', 'iteration step', 'return result']
order = ['0', '1', '2', '3', '4', '5', '6', '7', '8']

In [8]:
def is_in(df, y):
    for _idx, row in df.iterrows():
        if row["startHeight"] <= y <= row["stopHeight"]:
            return row["Name"]
    return "none"

In [9]:
cols = ["startHeight", "stopHeight", "startWidth", "stopWidth", "Name"]

relation_tensor = []
for idx, tmp_df in enumerate(df_array):
    relation = []
    
    for i in range(len(aoi_names)):
        relation.append([0]*len(aoi_names))
    
    
    for row_idx, row in tmp_df.iterrows():
        algo_name = row["Algorithm"]
        
        aoi_df = aoi_df_array[algo_name_array.tolist().index(algo_name)]
        click_data = row["ClickData"] 
        click_data = to_valid(click_data)
        labels = [is_in(aoi_df, coord) for coord in click_data]
        
        for i in range(len(labels)-1):
            from_node = aoi_names.index(labels[i])
            to_node = aoi_names.index(labels[i+1])
            
            relation[from_node][to_node] += 1
    relation_tensor.append(relation)

In [10]:
import numpy as np

for matrix in relation_tensor:
    for idx in range(len(matrix)):
        matrix[idx][idx] = 0
        
tensor = [np.array(matrix)[1:, 1:].tolist() for matrix in relation_tensor]
order = ['Main', 'Definition', 'Pre Calculation', 'If Condition', 'If Step', 'Iteration Condition', 'Iteration Step', 'Result']
#order = ['1', '2', '3', '4', '5', '6', '7', '8']

In [11]:
Chord(tensor[0], order, padding=0.1).to_html("tmp.html")
#Chord(tensor[1], order, padding=0.1).to_html("BI_general.html")

7 5 0
7 5 7
5 5 7
5 7 7
